# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235150063740                   -0.50    8.0         
  2   -7.250229469694       -1.82       -1.40    1.0   7.95ms
  3   -7.251161674284       -3.03       -2.20    1.0   8.10ms
  4   -7.251273675662       -3.95       -2.20    2.0   10.4ms
  5   -7.251332804028       -4.23       -2.66    1.0   8.46ms
  6   -7.251337879562       -5.29       -3.12    1.0   8.42ms
  7   -7.251338747177       -6.06       -3.73    2.0   9.72ms
  8   -7.251338788784       -7.38       -4.00    2.0   10.1ms
  9   -7.251338796576       -8.11       -4.37    1.0   8.55ms
 10   -7.251338798029       -8.84       -4.62    2.0   83.0ms
 11   -7.251338798606       -9.24       -5.10    1.0   9.29ms
 12   -7.251338798694      -10.06       -5.47    2.0   10.3ms
 13   -7.251338798701      -11.13       -5.75    1.0   8.43ms
 14   -7.251338798704      -11.58       -6.27    2.0   9.55ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.048258649185405816
[ Info: Arnoldi iteration step 2: normres = 0.6578353258841745
[ Info: Arnoldi iteration step 3: normres = 0.8542113303682541
[ Info: Arnoldi iteration step 4: normres = 0.3215092921019278
[ Info: Arnoldi iteration step 5: normres = 0.3965467753158967
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.43e-02, 7.07e-02, 3.05e-01, 2.41e-01, 6.92e-03)
[ Info: Arnoldi iteration step 6: normres = 0.3746127138809356
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.08e-02, 1.23e-01, 3.13e-01, 9.95e-02, 1.08e-01)
[ Info: Arnoldi iteration step 7: normres = 0.0903156710122267
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.83e-04, 1.36e-02, 2.28e-02, 3.76e-02, 6.99e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09423146625252836
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.99e-0